In [123]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import cv2
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

tf.compat.v1.disable_eager_execution()

In [124]:
flow_path = os.listdir('./datasets/flowers/')
flow_path

['daisy', 'dandelion', 'rose', 'sunflower', 'tulip']

In [125]:
path = './datasets/flowers/'

img_size = 64

image = []
label = []

for i in flow_path:
    data_path = path + str(i)
    fileName = [i for i in os.listdir(data_path) if i.endswith('jpg')]

    # print(fileName)

    for f in fileName:
        # reading the images
        img = cv2.imread(data_path + '/' + f)
        # pirnt(img)

        # resizing the images
        img = cv2.resize(img, (img_size, img_size))

        image.append(img)
        label.append(i)


In [126]:
image = np.array(image)
image = image.astype('float16') / 255

image

array([[[[0.545   , 0.5527  , 0.5527  ],
         [0.5884  , 0.5845  , 0.5845  ],
         [0.6     , 0.596   , 0.596   ],
         ...,
         [0.655   , 0.6353  , 0.639   ],
         [0.596   , 0.604   , 0.604   ],
         [0.6     , 0.6     , 0.6     ]],

        [[0.5137  , 0.533   , 0.537   ],
         [0.565   , 0.569   , 0.5728  ],
         [0.678   , 0.655   , 0.655   ],
         ...,
         [0.639   , 0.6196  , 0.6235  ],
         [0.6     , 0.608   , 0.608   ],
         [0.5845  , 0.5845  , 0.5845  ]],

        [[0.4626  , 0.4902  , 0.4941  ],
         [0.537   , 0.549   , 0.541   ],
         [0.604   , 0.604   , 0.604   ],
         ...,
         [0.612   , 0.612   , 0.612   ],
         [0.604   , 0.6157  , 0.6157  ],
         [0.5566  , 0.5605  , 0.565   ]],

        ...,

        [[0.0902  , 0.1726  , 0.1608  ],
         [0.098   , 0.1804  , 0.1686  ],
         [0.1412  , 0.2     , 0.1921  ],
         ...,
         [0.4785  , 0.4863  , 0.502   ],
         [0.4626  , 0.

In [127]:
image.shape

(4317, 64, 64, 3)

In [128]:

X = image
y = label

# change the labels to numeirc values
y_label_encoder = LabelEncoder()
y = y_label_encoder.fit_transform(y)

y

array([0, 0, 0, ..., 4, 4, 4], dtype=int64)

In [129]:
# oneHotEncodering the y

y = y.reshape(-1, 1)

onehotencoder = OneHotEncoder()

y = onehotencoder.fit_transform(y)

y = y.toarray()

y

array([[1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.]])

In [130]:
X, y = shuffle(X, y, random_state=42)

In [131]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)

In [132]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((3453, 64, 64, 3), (864, 64, 64, 3), (3453, 5), (864, 5))

#### for processing we should change the shape of images to one dimensional arrays that why we multiply the (64, 64, 3) that it is equal to 12288

In [133]:
X_train = X_train.reshape(3453, 12288)

X_test = X_test.reshape(864, 12288)

In [134]:
X_train.shape, X_test.shape

((3453, 12288), (864, 12288))

In [135]:
lr = .001
epoch = 100
n_class = 5
n_dim = 64

x = tf.compat.v1.placeholder(tf.float16, [None, 12288])
w = tf.Variable(tf.cast(tf.zeros([n_class, 12288]), tf.float16))
b = tf.Variable(tf.cast(tf.zeros([n_class]), tf.float16))
y_= tf.compat.v1.placeholder(tf.float16, [None, n_class])


In [136]:
init = tf.compat.v1.global_variables_initializer()

In [137]:

pred = tf.nn.softmax(tf.matmul(x, tf.transpose(w)) + b) # preceptron


In [140]:
cost_function = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y_))

In [142]:
training_steps = tf.compat.v1.train.GradientDescentOptimizer(lr).minimize(cost_function)

In [143]:
sess = tf.compat.v1.Session()
sess.run(init)
cost_history = []

In [144]:
for i in range(epoch):
    sess.run(training_steps, feed_dict={x:X_train, y_:y_train})
    cost = sess.run(cost_function, feed_dict={x:X_train, y_:y_train})

    cost_history = np.append(cost_history, cost)

    print('epoch: ', i , 'cost: ', cost)

y_pred = sess.run(pred, feed_dict={x:X_test})

epoch:  0 cost:  1.62
epoch:  1 cost:  1.62
epoch:  2 cost:  1.619
epoch:  3 cost:  1.617
epoch:  4 cost:  1.616
epoch:  5 cost:  1.615
epoch:  6 cost:  1.614
epoch:  7 cost:  1.612
epoch:  8 cost:  1.61
epoch:  9 cost:  1.61
epoch:  10 cost:  1.608
epoch:  11 cost:  1.607
epoch:  12 cost:  1.606
epoch:  13 cost:  1.606
epoch:  14 cost:  1.604
epoch:  15 cost:  1.604
epoch:  16 cost:  1.603
epoch:  17 cost:  1.602
epoch:  18 cost:  1.602
epoch:  19 cost:  1.599
epoch:  20 cost:  1.599
epoch:  21 cost:  1.599
epoch:  22 cost:  1.599
epoch:  23 cost:  1.598
epoch:  24 cost:  1.597
epoch:  25 cost:  1.597
epoch:  26 cost:  1.597
epoch:  27 cost:  1.597
epoch:  28 cost:  1.597
epoch:  29 cost:  1.597
epoch:  30 cost:  1.596
epoch:  31 cost:  1.596
epoch:  32 cost:  1.595
epoch:  33 cost:  1.595
epoch:  34 cost:  1.595
epoch:  35 cost:  1.595
epoch:  36 cost:  1.595
epoch:  37 cost:  1.595
epoch:  38 cost:  1.595
epoch:  39 cost:  1.595
epoch:  40 cost:  1.594
epoch:  41 cost:  1.594
epoch: